## Molkit

In [1]:
# import libraries
import numpy as np
from moha import HamHeisenberg

J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]
                 ])

mu = np.zeros(2)

ham = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham.generate_one_body_integral(dense=True)
two_body = ham.generate_two_body_integral(dense=True)

In [ ]:
import numpy as np
from moha import HamHeisenberg
from moha.molkit.hamiltonians import MolHam
from moha.molkit.utils.tools import from_geminal
from moha.molkit.utils.spinops import antisymmetrize_two_body

def from_geminal(two_body_gem, n_orb):
    """
    Inverse of MolHam.to_geminal().

    Parameters
    ----------
    two_body_gem : (n_gem, n_gem) ndarray
        Matrix in the geminal basis.
    n_orb : int
        Number of spin orbitals.

    Returns
    -------
    V : (n_orb, n_orb, n_orb, n_orb) ndarray
        Fully antisymmetrised two-electron tensor V_{ijkl}.
    """
    n_gem = n_orb * (n_orb - 1) // 2
    if two_body_gem.shape != (n_gem, n_gem):
        raise ValueError(f"Shape mismatch: got {two_body_gem.shape}, expected ({n_gem},{n_gem})")

    # Generate flattened pair list exactly like to_geminal
    pairs = [(i, j) for i in range(n_orb) for j in range(i + 1, n_orb)]
    V = np.zeros((n_orb, n_orb, n_orb, n_orb))

    for A, (i, j) in enumerate(pairs):
        for B, (k, l) in enumerate(pairs):
            val = 0.25 * two_body_gem[A, B]  # undo the factor 0.5 * 4 = 2

            # Apply antisymmetric filling
            V[i, j, k, l] = val
            V[j, i, k, l] = -val
            V[i, j, l, k] = -val
            V[j, i, l, k] = val
            V[k, l, i, j] = val
            V[l, k, i, j] = -val
            V[k, l, j, i] = -val
            V[l, k, j, i] = val

    return V


J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]])
mu = np.zeros(2)

ham_hei  = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham_hei.generate_one_body_integral(dense=True)
two_body = ham_hei.generate_two_body_integral(dense=True)
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)

h1_spin, h2_spin = Molecular_Hamiltonian.spinize_H()
h2_spin = antisymmetrize_two_body(h2_spin)

G = MolHam.to_geminal(h2_spin)   
n_orb = h2_spin.shape[0]

V = from_geminal(G, n_orb = n_orb)
print("Max error:", np.max(np.abs(h2_spin - V)))


Max error: 1.0
